<a href="https://colab.research.google.com/github/vijaidjearam/NSEstockanalysis/blob/main/backtest_01032024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation of Libraries Required for the RunTime

---



In [ ]:
# @title
! pip install nsepython
! pip install nselib

# Import all the required libraries

---



In [2]:
# @title
import pandas as pd
from nsepython import *
from datetime import datetime, timedelta, date
from nselib import capital_market
from dateutil import parser
import math
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


# Dependent functions required for the main module

---



In [4]:
# @title
def get_index_history_lastweek(symbol):
    start_date = (datetime.today() - timedelta(days=7)).strftime("%m/%d/%Y")
    end_date = datetime.now().strftime("%m/%d/%Y")
    df = pd.DataFrame(index_history(symbol,start_date,end_date))
    print(df)
def get_index_history(symbol,start_date,end_date):
    data = capital_market.index_data(index=symbol,from_date=start_date,to_date=end_date)
    print (data)
    return data
def gethighofstartdate(df,symbol,start_date,end_date):
    filt = (df['TIMESTAMP'] == start_date)
    try:
        result = df.loc[filt,'HIGH_INDEX_VAL'].values[0]
        return float(result)
    except IndexError:
        return "NA"
    except ValueError as ve:
        return ve
def getlowofstartdate(df,symbol,start_date,end_date):
    filt = (df['TIMESTAMP'] == start_date)
    try:
        result = df.loc[filt,'LOW_INDEX_VAL'].values[0]
        return float(result)
    except IndexError:
        return "NA"
    except ValueError as ve:
        return ve
def gethighofenddate(df,symbol,start_date,end_date):
    filt = (df['TIMESTAMP'] == end_date)
    try:
        result = df.loc[filt,'HIGH_INDEX_VAL'].values[0]
        return float(result)
    except IndexError:
        return "NA"
    except ValueError as ve:
        return ve
def getlowofenddate(df,symbol,start_date,end_date):
    filt = (df['TIMESTAMP'] == end_date)
    try:
        result = df.loc[filt,'LOW_INDEX_VAL'].values[0]
        return float(result)
    except IndexError:
        return "NA"
    except ValueError as ve:
        return ve
def derivativehistorycallgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice):
    optionType = "CE"
    start_date = datetime.strptime(start_date,"%d-%m-%Y")
    start_date = start_date.strftime("%d-%m-%Y")
    end_date = datetime.strptime(end_date,"%d-%m-%Y")
    end_date = end_date.strftime("%d-%m-%Y")
    expiry_date = datetime.strptime(expiry_date,"%d-%m-%Y")
    expiry_date = expiry_date.strftime("%d-%b-%Y")
    try:
        df = pd.DataFrame(derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,optionType))
        result = (df['FH_TRADE_LOW_PRICE'].min())
        return float(result)
    except:
        return "NA"
def derivativehistoryputgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,):
    optionType = "PE"
    start_date = datetime.strptime(start_date,"%d-%m-%Y")
    start_date = start_date.strftime("%d-%m-%Y")
    end_date = datetime.strptime(end_date,"%d-%m-%Y")
    end_date = end_date.strftime("%d-%m-%Y")
    expiry_date = datetime.strptime(expiry_date,"%d-%m-%Y")
    expiry_date = expiry_date.strftime("%d-%b-%Y")
    try:
        df = pd.DataFrame(derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,optionType))
        result = (df['FH_TRADE_LOW_PRICE'].min())
        return float(result)
    except:
        return "NA"
def round_up_to_base(x, base=50):
    return x + (base - x) % base
def round_down_to_base(x, base=50):
    return x - (x % base)

def get_end_strike_value(index_symbol,start_date,end_date,expiry_date,instrumentType):
    symbol = index_symbol
    df = get_index_history(symbol,start_date,end_date)
    highofstartdate = gethighofstartdate(df,symbol,start_date,end_date)
    print('highofstartdate : '+ str(highofstartdate))
    lowofstartdate = getlowofstartdate(df,symbol,start_date,end_date)
    print('lowofstartdate: '+ str(lowofstartdate))
    highofenddate = gethighofenddate(df,symbol,start_date,end_date)
    print('highofenddate: '+ str(highofenddate))
    lowofenddate = getlowofenddate(df,symbol,start_date,end_date)
    print('lowofenddate : '+ str(lowofenddate))
    maxof2days = max(highofstartdate,highofenddate)
    print('maxof2days: '+ str(maxof2days))
    minof2days = min(lowofstartdate,lowofenddate)
    print('minof2days :' + str(minof2days))
    bufferhigh = round(maxof2days * 1.0015)
    print('bufferhigh : '+ str(bufferhigh))
    bufferlow = round(minof2days * 0.9985)
    print('bufferlow : '+ str(bufferlow))
    callendstrike = round_down_to_base(bufferlow)
    print('callendstrike : '+str(callendstrike))
    putendstrike = round_up_to_base(bufferhigh)
    print('putendstrike : '+str(putendstrike))
    return callendstrike,putendstrike
def get_call_entrystrike(derivative_symbol,start_date,end_date,instrumentType,expiry_date,callendstrike):
    # calculate CallEntryStrike 1st row
    symbol = derivative_symbol
    strike = []
    premium = []
    twodll = []
    strike.append(callendstrike)
    temppremium = callendstrike * (0.85/100)
    premium.append(temppremium)
    temptwodll = float(derivativehistorycallgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice=strike[0]))
    twodll.append(temptwodll)
     # Call Entry strke 2....n rows
    for i in range(1,9):
        if twodll[-1] >= premium[-1]:
            strike.append(strike[-1]+50)
            premium.append(strike[-1]* (0.85/100))
            twodll.append(derivativehistorycallgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice=strike[-1]))
        else:
            break
    calldf = pd.DataFrame({"strike":strike,"premium":premium,"twodll":twodll})
    calldf['diff']=round(calldf['twodll']-calldf['premium'],2)
    print (calldf)
    leastdiff = calldf.loc[calldf['diff']>0,'diff'].min()
    leastdiff = round(leastdiff,2)
    print ('leastdiff : '+str(leastdiff))
    if math.isnan(leastdiff):
        callentrystrike = 'NA'
        calltwodll = 'NA'
    else:
        filt = (calldf['diff'] == leastdiff)
        callentrystrike = calldf.loc[filt,'strike'].values[0]
        calltwodll = calldf.loc[filt,'twodll'].values[0]
    print("Call entry strike : ",callentrystrike)
    print("Call entry twoDLL : ",calltwodll)
    return callentrystrike,calltwodll
def get_put_entrystrike(derivative_symbol,start_date,end_date,instrumentType,expiry_date,putendstrike):
    # finding PutEntryStrike 1st row
    symbol = derivative_symbol
    strike = []
    premium = []
    twodll = []
    strike.append(putendstrike)
    temppremium = putendstrike * (0.85/100)
    premium.append(temppremium)
    temptwodll = derivativehistoryputgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice=strike[0])
    twodll.append(temptwodll)
    # Put Entry strke 2....n rows
    for i in range(1,9):
        if twodll[-1] >= premium[-1]:
            strike.append(strike[-1]-50)
            premium.append(strike[-1]* (0.85/100))
            twodll.append(derivativehistoryputgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice=strike[-1]))
        else:
            break
    putdf = pd.DataFrame({"strike":strike,"premium":premium,"twodll":twodll})
    putdf['diff']=round(putdf['twodll']-putdf['premium'],2)
    print (putdf)
    leastdiff = putdf.loc[putdf['diff']>0,'diff'].min()
    leastdiff = round(leastdiff,2)
    print ('leastdiff : '+str(leastdiff))
    if math.isnan(leastdiff):
        putentrystrike = 'NA'
        puttwodll = 'NA'
    else:
        filt = (putdf['diff'] == leastdiff)
        putentrystrike = putdf.loc[filt,'strike'].values[0]
        puttwodll = putdf.loc[filt,'twodll'].values[0]
    print("Put entry strike : ",putentrystrike)
    print("Put entry twoDLL : ",puttwodll)
    return putentrystrike,puttwodll

def expiry_history_dates(symbol,start_date,end_date):
  nsefetch_url = "https://www.nseindia.com/api/historical/fo/derivatives/meta?&from="+start_date+"&to="+end_date+"&symbol="+symbol+""
  payload = nsefetch(nsefetch_url)
  return payload
def spot_price(symbol,start_date,end_date):
  data = capital_market.index_data(index=symbol,from_date=start_date,to_date=end_date)
  filt = (data['TIMESTAMP'] == end_date)
  result = data.loc[filt,'CLOSE_INDEX_VAL'].values[0]
  return float(result)

def callentry_YN(symbol,start_date,end_date,expiry_date,strikePrice,CallEntry):
    optionType="CE"
    instrumentType = "options"
    expiry_date = datetime.strptime(expiry_date,"%d-%m-%Y")
    expiry_date = expiry_date.strftime("%d-%b-%Y")
    df = pd.DataFrame(derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,optionType))
    end_date = datetime.strptime(end_date,"%d-%m-%Y")
    end_date = end_date.strftime("%d-%b-%Y")
    filt = (df['FH_TIMESTAMP'] == end_date)
    low = float(df.loc[filt,'FH_TRADE_LOW_PRICE'].values[0])
    if low < CallEntry:
      return "Y"
    else:
      return "N"

def putentry_YN(symbol,start_date,end_date,expiry_date,strikePrice,PutEntry):
    optionType="PE"
    instrumentType = "options"
    expiry_date = datetime.strptime(expiry_date,"%d-%m-%Y")
    expiry_date = expiry_date.strftime("%d-%b-%Y")
    df = pd.DataFrame(derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,optionType))
    end_date = datetime.strptime(end_date,"%d-%m-%Y")
    end_date = end_date.strftime("%d-%b-%Y")
    filt = (df['FH_TIMESTAMP'] == end_date)
    low = float(df.loc[filt,'FH_TRADE_LOW_PRICE'].values[0])
    if low < CallEntry:
      return "Y"
    else:
      return "N"

def call_exit_price(symbol,start_date,end_date,expiry_date,strikePrice,CallTarget,CallStoploss):
    optionType="CE"
    instrumentType = "options"
    expiry_date = datetime.strptime(expiry_date,"%d-%m-%Y")
    expiry_date = expiry_date.strftime("%d-%b-%Y")
    df = pd.DataFrame(derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,optionType))
    for index, row in df.iterrows():
      print (row['FH_TRADE_LOW_PRICE'])
      if CallTarget <= float(row['FH_TRADE_LOW_PRICE']):
        call_exit_price = row['FH_TRADE_LOW_PRICE']
        call_exit_date = row['FH_TIMESTAMP']
        return call_exit_price,call_exit_date
      elif CallStoploss >= float(row['FH_TRADE_HIGH_PRICE']):
        call_exit_price = row['FH_TRADE_HIGH_PRICE']
        call_exit_date = row['FH_TIMESTAMP']
        return call_exit_price,call_exit_date
      else:
        call_exit_price = 'NA'
        call_exit_date = 'NA'
        return call_exit_price,call_exit_date
def put_exit_price(symbol,start_date,end_date,expiry_date,strikePrice,CallTarget,CallStoploss):
    optionType="PE"
    instrumentType = "options"
    expiry_date = datetime.strptime(expiry_date,"%d-%m-%Y")
    expiry_date = expiry_date.strftime("%d-%b-%Y")
    df = pd.DataFrame(derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,optionType))
    for index, row in df.iterrows():
      print (row['FH_TRADE_LOW_PRICE'])
      if CallTarget <= float(row['FH_TRADE_LOW_PRICE']):
        call_exit_price = row['FH_TRADE_LOW_PRICE']
        call_exit_date = row['FH_TIMESTAMP']
        return call_exit_price,call_exit_date
      elif CallStoploss >= float(row['FH_TRADE_HIGH_PRICE']):
        call_exit_price = row['FH_TRADE_HIGH_PRICE']
        call_exit_date = row['FH_TIMESTAMP']
        return call_exit_price,call_exit_date
      else:
        call_exit_price = 'NA'
        call_exit_date = 'NA'
        return call_exit_price,call_exit_date
def backtest(index_symbol,derivative_symbol,start_date,end_date,instrumentType,expiry_date):
    print(start_date)
    print(end_date)
    print(expiry_date)
    expiry_date_temp = expiry_history_dates(derivative_symbol,start_date,end_date)
    expiry_date_list = expiry_date_temp['expiryDatesByInstrument']['OPTIDX']
    print(expiry_date_list)
    callendstrike,putendstrike = get_end_strike_value(index_symbol,start_date,end_date,expiry_date,instrumentType)
    expiry_date = datetime.strptime(expiry_date,"%d-%m-%Y")
    print(expiry_date)
    expiry_date =expiry_date.strftime("%d-%b-%Y")
    print(expiry_date)
    index = expiry_date_list.index(expiry_date)
    #The expiry dates is a list that starts with the selected expiry date and ends with the end of the expiry_date_list
    expiry_dates = expiry_date_list[index:-1]
    callentrystrike = 'NA'
    putentrystrike = 'NA'
    for item in expiry_dates:
      if callentrystrike == 'NA':
        expiry_date = datetime.strptime(item,"%d-%b-%Y")
        expiry_date = expiry_date.strftime("%d-%m-%Y")
        print ("--------Calculating Call entry strike for expiry date : "+expiry_date)
        callentrystrike,calltwodll = get_call_entrystrike(derivative_symbol,start_date,end_date,instrumentType,expiry_date,callendstrike)

    Call_expiry_date = expiry_date
    CallEntry = 0.9 * calltwodll
    CallTarget = CallEntry * 0.25
    CallStoploss = CallEntry * 1.75
    print("CallEntry : "+str(CallEntry))
    print("CallTarget :" +str(CallTarget))
    print("CallStoploss : "+str(CallStoploss))


    for item in expiry_dates:
      if putentrystrike == 'NA':
        expiry_date = datetime.strptime(item,"%d-%b-%Y")
        expiry_date = expiry_date.strftime("%d-%m-%Y")
        print ("--------Calculating Put entry strike for expiry date : "+expiry_date)
        putentrystrike,puttwodll = get_put_entrystrike(derivative_symbol,start_date,end_date,instrumentType,expiry_date,putendstrike)

    put_expiry_date = expiry_date
    PutEntry = 0.9 * puttwodll
    PutTarget = PutEntry * 0.25
    PutStoploss = PutEntry * 1.75
    print("PutEntry : "+str(PutEntry))
    print("PutTarget :" +str(PutTarget))
    print("PutStoploss : "+str(PutStoploss))
    # Clearing all data for the next run
    start_date = ""
    end_date = ""
    expiry_date = ""
    expiry_date_list = ""
    expiry_date_picker = ""
    expiry_date_selected = ""
    return callentrystrike,calltwodll,Call_expiry_date,CallEntry,CallTarget,CallStoploss,putentrystrike,puttwodll,put_expiry_date,PutEntry,PutTarget,PutStoploss

Test Loop

In [ ]:
# @title
index_symbol = 'NIFTY 50'
derivative_symbol = "NIFTY"
start_date = "02-01-2024"
end_date = "03-01-2024"
expiry_date = "11-01-2024"
instrumentType = "options"
callentrystrike,calltwodll,Call_expiry_date,CallEntry,CallTarget,CallStoploss,putentrystrike,puttwodll,put_expiry_date,PutEntry,PutTarget,PutStoploss = backtest(index_symbol,derivative_symbol,start_date,end_date,instrumentType,expiry_date)
print(callentrystrike)
print(calltwodll)
print(Call_expiry_date)
print(CallEntry)
print(CallTarget)
print(CallStoploss)
print(putentrystrike)
print(puttwodll)
print(put_expiry_date)
print(PutEntry)
print(PutTarget)
print(PutStoploss)

#Main loop

In [ ]:
index_symbol = 'NIFTY 50'
derivative_symbol = "NIFTY"
instrumentType = "options"
worksheet = gc.open_by_key('1KWaQRcYOP_SiAT7wjsWDNvPtj9duiupsTILWr0QJk64').sheet1
#rows = worksheet.get_all_values()
#df = pd.DataFrame.from_records(rows[1:],columns=rows[0])
df = pd.DataFrame(worksheet.get_all_records())
for i in df.index:
  if df.loc[i,"call_entry_strike"]:
    pass
  else:
    x = i+2
    print ("###-------------------------------------------------------------------------------Processing Row : "+str(x)+"--------------------------------------------------------------------------------###")
    start_date = df.loc[i,"start_date"]
    end_date = df.loc[i,"end_date"]
    expiry_date = df.loc[i,"expiry_date"]
    execution_date = df.loc[i,"execution_date"]
    callentrystrike,calltwodll,Call_expiry_date,CallEntry,CallTarget,CallStoploss,putentrystrike,puttwodll,put_expiry_date,PutEntry,PutTarget,PutStoploss = backtest(index_symbol,derivative_symbol,start_date,end_date,instrumentType,expiry_date)
    worksheet.update_cell(x,7,float(callentrystrike))
    worksheet.update_cell(x,8,CallEntry)
    worksheet.update_cell(x,9,CallTarget)
    worksheet.update_cell(x,10,CallStoploss)
    worksheet.update_cell(x,11,Call_expiry_date)
    worksheet.update_cell(x,16,float(putentrystrike))
    worksheet.update_cell(x,17,PutEntry)
    worksheet.update_cell(x,18,PutTarget)
    worksheet.update_cell(x,19,PutStoploss)
    worksheet.update_cell(x,20,put_expiry_date)
    spot = spot_price(index_symbol,start_date,end_date)
    worksheet.update_cell(x,6,spot)
    callentry_yn = callentry_YN(derivative_symbol,start_date,execution_date,Call_expiry_date,callentrystrike,CallEntry)
    worksheet.update_cell(x,12,callentry_yn)
    call_ep,call_ed = call_exit_price(derivative_symbol,execution_date,Call_expiry_date,Call_expiry_date,callentrystrike,CallTarget,CallStoploss)
    worksheet.update_cell(x,13,call_ep)
    worksheet.update_cell(x,14,call_ed)
    putentry_yn = putentry_YN(derivative_symbol,start_date,execution_date,put_expiry_date,putentrystrike,PutEntry)
    worksheet.update_cell(x,21,putentry_yn)
    put_ep,put_ed = put_exit_price(derivative_symbol,execution_date,put_expiry_date,put_expiry_date,putentrystrike,PutTarget,PutStoploss)
    worksheet.update_cell(x,22,put_ep)
    worksheet.update_cell(x,23,put_ed)



```
# Change logs
```
--18/02/2024

Code requests

1. include date selection in index history ✅

2. get all expiry date,if leasediff nan ,try with next expiry date ✅

3. Entry=0.9*2dll; Target=Entry*0.25; Stoploss=Entry*1.75 ✅

--26/02/2024

1. Include functionality for back testing ✅

--28/02/2024

1. spot is close price of end date. (from https://www.nseindia.com/reports-indices-historical-index-data) ✅

2. execution date is user input data for trade execution.
3. Call entry (Y/N) go to https://www.nseindia.com/report-detail/fo_eq_security startdate and end date = execution date and the expiry date is the call-expiry-date
compare low call-entry price with execution day low
if excution day low < call_entry print yes
else:
non ✅
4. call-exit_price
 go to https://www.nseindia.com/report-detail/fo_eq_security
 start date = execution date
 end date = call_expiry_date
 expiry_date = call_expiry_date
 strike price = call_entry_strike
 type = CE
 check for the call target (eg : 45 or below) in  low
 if found then call exit price = call-target and set the start date to call_exit_date
 else check call stoploss (eg 320 or 320 above) in HIGH found then call exit price = call-stploss and coressonding start date to call-exit-date
 else
 NA ✅








